In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tensorflow.keras import callbacks as callbacks_
from tensorflow.keras import layers
from keras import models

2023-03-03 18:51:21.723475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 18:51:21.814781: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-03 18:51:22.267724: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/kannika/miniconda3/envs/vit-tf/lib/:/home/kannika/miniconda3/envs/vit-tf/lib/:/h

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [28]:
IMAGE_SIZE = 224 
BATCH_SIZE = 16
EPOCHS = 3
# EPOCHS = 4

In [3]:
DF = pd.read_csv('/home/kannika/codes_AI/Rheology2023/datasetMSDT_train_valid.csv')
DF_TRAIN = DF[DF['subset']=='train'].reset_index(drop=True)
print(DF_TRAIN.shape)
DF_VAL = DF[DF['subset']=='valid'].reset_index(drop=True)
print(DF_VAL.shape)

(19778, 9)
(3857, 9)


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='constant')

valid_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
## DataSet
train_generator = train_datagen.flow_from_dataframe(
        dataframe = DF_TRAIN,
        directory = None,
        x_col = 'pathimg',
        y_col = 'MSDT',
        target_size = (IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        color_mode= 'rgb',
        class_mode='raw')

val_generator = valid_datagen.flow_from_dataframe(
        dataframe = DF_VAL,
        directory = None,
        x_col = 'pathimg',
        y_col = 'MSDT',
        target_size = (IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        color_mode= 'rgb',
        class_mode='raw')

Found 19778 validated image filenames.
Found 3857 validated image filenames.


In [8]:
## Set TensorBoard 
from tensorflow.keras import callbacks
from keras.callbacks import Callback

root_logdir = f'/media/SSD/rheology2023/VitModel/Regression/tensorflow/ExpTest/R1/Mylogs_tensor/'
if not os.path.exists(root_logdir) :
    os.makedirs(root_logdir)

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir,run_id)
run_logdir = get_run_logdir()

tensorboard_cb = callbacks.TensorBoard(log_dir=run_logdir)

In [9]:
### ViT model with the functional ###
from vit_keras import vit, utils

vit_model = vit.vit_l32(
        image_size = IMAGE_SIZE,
        pretrained = True,
        include_top = False,
        pretrained_top = False
        )

2023-03-03 19:05:55.233270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 19:05:55.666914: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9517 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5


1226658854/1226658854 [==============================] - 197s 0us/step


/home/kannika/miniconda3/envs/vit-tf/lib/python3.8/site-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 12, 12 to 7, 7
  warnings.warn(


In [10]:
vit_model

In [12]:
vit_model.summary()

Model: "vit-l32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 7, 7, 1024)        3146752   
                                                                 
 reshape (Reshape)           (None, 49, 1024)          0         
                                                                 
 class_token (ClassToken)    (None, 50, 1024)          1024      
                                                                 
 Transformer/posembed_input   (None, 50, 1024)         51200     
 (AddPositionEmbs)                                               
                                                                 
 Transformer/encoderblock_0   ((None, 50, 1024),       12596224  
 (TransformerBlock)           (None, 16, None, None))      

In [14]:
x = vit_model.get_layer('ExtractToken').output
x

<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'ExtractToken')>

In [16]:
### add the tail layer ###  
Flatten_layer1 = layers.Flatten()(x)
BatchNormalization_layer1 = layers.BatchNormalization(name='BatchNormalization_1')(Flatten_layer1)
Dense_layer1 = layers.Dense(64, activation='gelu',name='Dense_regress')(BatchNormalization_layer1)
BatchNormalization_layer2 = layers.BatchNormalization(name='BatchNormalization_2')(Dense_layer1)
Dense_layer2 = layers.Dense(1, activation='linear',name='linear_regress')(BatchNormalization_layer2)

In [17]:
model = models.Model(inputs= vit_model.input, outputs=[Dense_layer2]) 
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 7, 7, 1024)        3146752   
                                                                 
 reshape_1 (Reshape)         (None, 49, 1024)          0         
                                                                 
 class_token (ClassToken)    (None, 50, 1024)          1024      
                                                                 
 Transformer/posembed_input   (None, 50, 1024)         51200     
 (AddPositionEmbs)                                               
                                                                 
 Transformer/encoderblock_0   ((None, 50, 1024),       12596224  
 (TransformerBlock)           (None, 16, None, None))        

In [18]:
##Freeze
print('This is the number of trainable layers '
          'before freezing the conv base:', len(model.trainable_weights))
for layer in vit_model.layers:
    layer.trainable = False
print('This is the number of trainable layers '
          'after freezing the conv base:', len(model.trainable_weights))
print('-'*80)
model.summary()

This is the number of trainable layers before freezing the conv base: 398
This is the number of trainable layers after freezing the conv base: 8
--------------------------------------------------------------------------------
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 7, 7, 1024)        3146752   
                                                                 
 reshape_1 (Reshape)         (None, 49, 1024)          0         
                                                                 
 class_token (ClassToken)    (None, 50, 1024)          1024      
                                                                 
 Transformer/posembed_input   (None, 50, 1024)         51200     
 (AddPositionEmbs)               

In [19]:
learning_rate = 1e-6

In [21]:
#optimizer = tfa.optimizers.RectifiedAdam(learning_rate = learning_rate)

In [23]:
from keras.optimizers import Adam

model.compile(optimizer = Adam(2e-6, decay=1e-6), 
              loss = 'mse', 
              metrics = ['mse'])

In [27]:

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = val_generator.n // val_generator.batch_size

root_base = '/media/SSD/rheology2023/VitModel/Regression/tensorflow/Exptest'
if not os.path.exists(root_base) :
    os.makedirs(root_base)
os.chdir(root_base)

## reduce_lr, checkpointer ต้องเปลี่ยนใหม่
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',  ##เปลี่ยนชื่อเป็น Validation_loss 
                                                 factor = 0.2,
                                                 patience = 2,
                                                 verbose = 1,
                                                 min_delta = 1e-5,
                                                 min_lr = 1e-6,
                                                 mode = 'max')


checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = './modelRegress_ViT_l32_Rheology.hdf5',
                                                  monitor = 'val_loss', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

callbacks = [reduce_lr, checkpointer]

In [30]:
root_logdir = f'{root_base}/run_log'  #เปลี่ยน R1_1 เปลี่ยนตาม fold

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir,run_id)
run_logdir = get_run_logdir()

tensorboard_cb = callbacks_.TensorBoard(log_dir = run_logdir)


def avoid_error(gen):
    while True:
        try:
            data, labels = next(gen)
            yield data, labels
        except:
            pass
        
model.fit(x = avoid_error(train_generator),
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_generator,
          validation_steps = STEP_SIZE_VALID,
          epochs = EPOCHS,
          callbacks = callbacks)

model.save('./models/modelRegress_ViT_l32_Rheology_R1.h5')

Epoch 1/3


2023-03-03 19:33:11.303448: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-03 19:33:12.144561: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-03 19:33:12.145524: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-03 19:33:12.145550: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:85] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-03-03 19:33:12.146396: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-03 19:33:12.146476: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1236/1236 [==============================] - ETA: 0s - loss: 118.1770 - mse: 118.1770
Epoch 1: val_loss improved from -inf to 134.56284, saving model to ./modelRegress_ViT_l32_Rheology.hdf5
1236/1236 [==============================] - 286s 218ms/step - loss: 118.1770 - mse: 118.1770 - val_loss: 134.5628 - val_mse: 134.5628 - lr: 2.0000e-06
Epoch 2/3
1236/1236 [==============================] - ETA: 0s - loss: 115.3145 - mse: 115.3145
Epoch 2: val_loss did not improve from 134.56284
1236/1236 [==============================] - 263s 213ms/step - loss: 115.3145 - mse: 115.3145 - val_loss: 113.5808 - val_mse: 113.5808 - lr: 2.0000e-06
Epoch 3/3
1236/1236 [==============================] - ETA: 0s - loss: 112.6565 - mse: 112.6565
Epoch 3: ReduceLROnPlateau reducing learning rate to 1e-06.

Epoch 3: val_loss did not improve from 134.56284
1236/1236 [==============================] - 264s 214ms/step - loss: 112.6565 - mse: 112.6565 - val_loss: 101.6923 - val_mse: 101.6923 - lr: 2.0000e-06


In [13]:
### ViT model with the functional ###
from vit_keras import vit, utils

vit_model = vit.vit_l32(
        image_size = IMAGE_SIZE,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 14)
vit_model.summary()

Model: "vit-l32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 7, 7, 1024)        3146752   
                                                                 
 reshape_1 (Reshape)         (None, 49, 1024)          0         
                                                                 
 class_token (ClassToken)    (None, 50, 1024)          1024      
                                                                 
 Transformer/posembed_input   (None, 50, 1024)         51200     
 (AddPositionEmbs)                                               
                                                                 
 Transformer/encoderblock_0   ((None, 50, 1024),       12596224  
 (TransformerBlock)           (None, 16, None, None))      